# 🍃 Wind Direction

This is just a demonstration of the different (wind) vector directions that can be calculated. Just have a look at the plots at the bottom, don't bother the boilerplate code.

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import itertools

In [ ]:
import parmesan
from parmesan.units import units
from parmesan.wind import wind_direction

In [ ]:
plt.rcParams["figure.figsize"] = (10, 10)
plt.rcParams["axes.grid"] = True
plt.rcParams["axes.axisbelow"] = True
plt.rcParams["font.size"] = 20
plt.rcParams["legend.fontsize"] = "medium"
plt.rcParams["font.family"] = "monospace"

In [ ]:
def arrow_grid(**arrow_props):
    # Set up figure and axis
    fig, ax = plt.subplots()
    ax.set_xlim((-1.7, 1.7))
    ax.set_ylim(ax.get_xlim())
    ax.set_aspect("equal")
    ax.set_frame_on(False)
    ax.tick_params(
        left=False, bottom=False, labelleft=False, labelbottom=False
    )

    # Grid and tick locations
    locator = matplotlib.ticker.MultipleLocator(0.5)
    ax.xaxis.set_major_locator(locator)
    ax.yaxis.set_major_locator(locator)

    # Axis arrows
    ax.arrow(-1.5, 0, 3, 0, facecolor="#444444", **arrow_props)
    ax.arrow(0, -1.5, 0, 3, facecolor="#444444", **arrow_props)

    return fig, ax

In [ ]:
def show_angle(
    angle,
    ax,
    radius,
    angle_kwargs=tuple(),
    **kwargs,
):
    kwargs.setdefault(
        "label",
        "{:>03.0f~P} | {}".format(
            angle.to("degrees"),
            " ".join(f"{k}={str(v):5s}" for k, v in angle_kwargs.items()),
        ),
    )
    angle = angle.to("degree").m
    ax.add_patch(
        matplotlib.patches.Arc(
            (0, 0),
            width=radius * 2,
            height=radius * 2,
            angle=(0 if angle_kwargs.get("math_origin") else 90)
            - (angle if angle_kwargs.get("clockwise") else 0),
            theta1=0,
            theta2=angle,
            **kwargs,
        )
    )

In [ ]:
def draw_example_vector(
    x, y, ax, label="(x,y)", invertedlabel="inverted", **arrow_props
):
    ax.arrow(
        0,
        0,
        x,
        y,
        facecolor="red",
        **{**arrow_props, "width": 0.08, "head_width": 0.2},
    )
    ax.text(x, y, label)
    ax.arrow(
        -x,
        -v,
        x,
        y,
        alpha=0.2,
        **{
            "facecolor": "red",
            **arrow_props,
            "width": 0.08,
            "head_width": 0.2,
        },
    )
    ax.text(-x, -y, invertedlabel, ha="right")

In [ ]:
arrow_props = dict(
    width=0.05,
    head_width=0.15,
    edgecolor="none",
    length_includes_head=True,
    zorder=10,
)

## The `wind_direction()` function

The `wind_direction()` function can be used to calculate the meteorological wind direction.

In [ ]:
fig, ax = arrow_grid(**arrow_props)
ax.text(1.6, 0, "E", ha="center", va="center")
ax.text(0, -1.6, "S", ha="center", va="center")
ax.text(0, 1.6, "N", ha="center", va="center")
ax.text(-1.6, 0, "W", ha="center", va="center")

u, v = 0.8, -1.1
draw_example_vector(
    x=u,
    y=v,
    label="wind vector\n(u,v)",
    invertedlabel="origin\nof the wind",
    ax=ax,
    **arrow_props
)

show_angle(
    ax=ax,
    angle=wind_direction(u=u, v=v),
    angle_kwargs=dict(inverted=True, clockwise=True, math_origin=False),
    linestyle="dotted",
    color="royalblue",
    radius=0.9,
    label="meteorological\nwind direction",
    linewidth=3,
)


# legend outside of the axis
ax.legend(bbox_to_anchor=(1, 0.8), frameon=False)

## The `vector.angle()` function

This demonstrates the difference angles calculatable using the `vector.angle()` function:

In [ ]:
fig, ax = arrow_grid(**arrow_props)
ax.text(1.6, 0, "x", ha="center", va="center")
ax.text(0, 1.6, "y", ha="center", va="center")

x, y = 0.8, -1.1
draw_example_vector(x=x, y=y, label="(x,y)", ax=ax, **arrow_props)


radius = itertools.count(start=1, step=-0.11)
color = itertools.cycle(("green", "orange", "blue", "red", "black", "violet"))
linestyle = itertools.cycle(
    ("solid", "dashed", "dotted", (0, (3, 1, 1, 1)), (0, (1, 1)))
)


# angle annotations
for angle_kwargs in parmesan.utils.all_argument_combinations(
    {k: (True, False) for k in ("clockwise", "inverted", "math_origin")}
):
    show_angle(
        angle=parmesan.vector.angle(x, y, **angle_kwargs),
        ax=ax,
        angle_kwargs=angle_kwargs,
        radius=next(radius),
        color=next(color),
        linestyle=next(linestyle),
        linewidth=3,
    )


# legend outside of the axis
plt.rcParams["legend.fontsize"] = "xx-small"
ax.legend(bbox_to_anchor=(1, 0.8), frameon=False)
fig.subplots_adjust(left=0, right=0.5)